In [1]:
import pandas as pd
import numpy as np

In [2]:
R1 = pd.read_csv("H1WG.SCP1.H1.R1_5kbWindowsVsControl_Calculations.txt", sep='\t')
R1['FoldChange']= R1["log(enhancerSeq/control)"]
R1.dropna(0,inplace=True)
R1.head()

,chrom,start,end,cumulative_coverage,mean_coverage,control_mean_coverage,log(enhancerSeq/control),FoldChange
1,chr1,5000,10000,3,0.0006,0.0056,-0.970037,-0.970037
2,chr1,10000,15000,471865,94.3730,37.3234,0.402867,0.402867
3,chr1,15000,20000,454345,90.8690,37.3428,0.386209,0.386209
4,chr1,20000,25000,307099,61.4198,27.5938,0.347497,0.347497
5,chr1,25000,30000,107159,21.4318,10.9186,0.292892,0.292892


In [3]:
R1_10th = np.percentile(R1["FoldChange"],90)
R1_top10 = R1[R1.FoldChange >= R1_10th]
R1_top10.head()

,chrom,start,end,cumulative_coverage,mean_coverage,control_mean_coverage,log(enhancerSeq/control),FoldChange
17,chr1,85000,90000,89353,17.8706,4.5760,0.591653,0.591653
53,chr1,265000,270000,9393,1.8786,0.4848,0.588272,0.588272
68,chr1,340000,345000,14139,2.8278,0.7244,0.591470,0.591470
106,chr1,530000,535000,461152,92.2304,23.8594,0.587215,0.587215
193,chr1,965000,970000,410712,82.1424,20.3380,0.606259,0.606259


In [68]:
needMerging = [] #this ends up being an array of the index of values in the R1_top10 index, which is non-consecutive b/c sliced from the R1 index
okBins = []
mergedR1 = pd.DataFrame(columns=['chrom','start','end','cumulative_coverage','mean_coverage','control_mean_coverage','log(enhancerSeq/control)','FoldChange'])
for i in range(len(R1_top10.index)):
    if i==0:
        pass
    else:
        if int(R1_top10.index[i]) == int(R1_top10.index[i-1]+1):
            #print("Bordering Bins!\t" + str(i)) #lots of bordering bins
            needMerging.append(i)
for i in R1_top10.index:
    if i not in needMerging:
        okBins.append(i)
        
print(str(len(needMerging)) + " bins need to be merged.")
print(str(len(okBins)) + " bins are fine.")
print(len(R1_top10.index))

7466 bins need to be merged.
6311 bins are fine.
57126


In [61]:
for i in range(len(needMerging)):
    if i == 0:
        pass
    else:
        secondRow = R1_top10.loc[R1_top10.index[needMerging[i]]]
        firstRow = R1_top10.loc[R1_top10.index[needMerging[i]-1]]
        
        #get values for the new row
        mergedChrom = secondRow.chrom
        mergedStart = firstRow.start
        mergedEnd = secondRow.end
        mergedCumCoverage = (secondRow.cumulative_coverage + firstRow.cumulative_coverage)/float(2)
        mergedMeanCoverage = (secondRow.mean_coverage + firstRow.mean_coverage)/float(2)
        mergedCtrlMeanCoverage = (secondRow.control_mean_coverage + firstRow.control_mean_coverage)/float(2)
        mergedLogEnhancer = np.log10(mergedMeanCoverage/mergedCtrlMeanCoverage) #could drop this row
        mergedFoldChange = mergedLogEnhancer
        
        #add to the dataframe
        mergedR1 = mergedR1.append({'chrom':mergedChrom,'start':mergedStart,'end':mergedEnd,'cumulative_coverage':mergedCumCoverage, 'mean_coverage':mergedMeanCoverage,'control_mean_coverage':mergedCtrlMeanCoverage,'log(enhancerSeq/control)':mergedLogEnhancer,'FoldChange':mergedFoldChange}, ignore_index=True)
        
    

In [63]:
for i in range(len(R1_top10.index)):
    if i==0:
        mergedR1 = mergedR1.append(R1_top10.loc[R1_top10.index[i]])
    else:
        if (int(R1_top10.index[i]) != int(R1_top10.index[i-1]+1)):
            #print("Bordering Bins!\t" + str(i)) #lots of bordering bins
            mergedR1 = mergedR1.append(R1_top10.loc[R1_top10.index[i]])

In [64]:
mergedR1.head()

,chrom,start,end,cumulative_coverage,mean_coverage,control_mean_coverage,log(enhancerSeq/control),FoldChange
0,chr1,4870000.0,4880000.0,67582.0,13.5164,3.4011,0.599242,0.599242
1,chr1,5245000.0,5255000.0,146138.0,29.2276,7.1385,0.612186,0.612186
2,chr1,5680000.0,5690000.0,172164.5,34.4329,8.4052,0.612426,0.612426
3,chr1,9480000.0,9490000.0,220383.0,44.0766,10.8717,0.607911,0.607911
4,chr1,9485000.0,9495000.0,232434.0,46.4868,11.4757,0.607550,0.607550


In [65]:
len(R1_top10)-len(mergedR1)

1